In [1]:
pip install sparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00


In [2]:
import time
import dask.array as da
import numpy as np
import sys
from dask.distributed import Client
from dask import compute
import sparse
import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date




In [3]:
from dask.distributed import Client
import pandas as pd




In [4]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=3d8b19d62aa529a4dacb0951af47bb93f827566bbd2960503ecb2acb65cb870d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Sentiment Detection") \
    .master("local[*]") \
    .config("spark.executor.memory", "10g") \
    .config("spark.driver.memory", "10g") \
    .config("spark.memory.offHeap.enabled", True) \
    .config("spark.memory.offHeap.size", "10g") \
    .config("spark.default.parallelism", "200") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC") \
    .config("spark.serializer", "org.apache.spark.serializer.JavaSerializer") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

schema = StructType([
    StructField("marketplace", StringType(), True),
    StructField("customer_id", StringType(), True),  # Change to IntegerType if applicable
    StructField("review_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("product_parent", StringType(), True),
    StructField("product_title", StringType(), True),
    StructField("product_category", StringType(), True),
    StructField("star_rating", IntegerType(), True),
    StructField("helpful_votes", IntegerType(), True),
    StructField("total_votes", IntegerType(), True),
    StructField("vine", StringType(), True),
    StructField("verified_purchase", StringType(), True),
    StructField("review_headline", StringType(), True),
    StructField("review_body", StringType(), True),
    StructField("review_date", DateType(), True)
])

In [8]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder \
    .appName("Load Multiple TSV Files") \
    .getOrCreate()

# List of file paths
data_paths = [
    '/content/drive/MyDrive/archive/amazon_reviews_us_Apparel_v1_00.tsv',
    '/content/drive/MyDrive/archive/amazon_reviews_us_Automotive_v1_00.tsv',
    '/content/drive/MyDrive/archive/amazon_reviews_us_Baby_v1_00.tsv',
    '/content/drive/MyDrive/archive/amazon_reviews_us_Beauty_v1_00.tsv',
    '/content/drive/MyDrive/archive/amazon_reviews_us_Books_v1_02.tsv',
    '/content/drive/MyDrive/archive/amazon_reviews_us_Camera_v1_00.tsv',
    '/content/drive/MyDrive/archive/amazon_reviews_us_Electronics_v1_00.tsv',
    '/content/drive/MyDrive/archive/amazon_reviews_us_Furniture_v1_00.tsv',
    '/content/drive/MyDrive/archive/amazon_reviews_us_Sports_v1_00.tsv',
    '/content/drive/MyDrive/archive/amazon_reviews_us_Grocery_v1_00.tsv',
    '/content/drive/MyDrive/archive/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv',
    '/content/drive/MyDrive/archive/amazon_reviews_us_Music_v1_00.tsv'
]

# Read and sample from each dataset
sampled_dfs = []
for path in data_paths:
    df = spark.read.option("treatEmptyValuesAsNulls", "true").option("sep", "\t").csv(path, schema=schema, sep='\t', header=True)
    sampled_df = df.limit(17000)  # Takes the first 17,000 rows
    sampled_dfs.append(sampled_df)

# Union all the sampled dataframes into one
df = sampled_dfs[0]
for dataframe in sampled_dfs[1:]:
    df = df.union(dataframe)

# Show some data
df.show()

# You can now work with 'final_df' which contains 17,000 rows from each file

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [9]:
from pyspark.sql.functions import col, lower, regexp_replace, concat_ws, udf, substring, row_number

# Remove duplicates
df = df.dropDuplicates()

# Handle missing values for both review_body and review_headline simultaneously
df = df.na.fill({
    "review_body": "No review text",
    "review_headline": "No review headline"
})

# Concatenate cleaned review headline and cleaned review text
df = df.withColumn("Full_Review", concat_ws(". ", "review_headline", "review_body"))

# Truncate full_text to the first 512 characters
df = df.withColumn("Full_Review", substring(col("Full_Review"), 1, 512))

df = df.drop("marketplace", "review_id", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_body","review_headline", "review_date", "product_parent","Full_Review")

df.cache()

DataFrame[customer_id: string, product_id: string, product_title: string, product_category: string, star_rating: int]

In [10]:
df = df.repartition(200)
df.cache()

DataFrame[customer_id: string, product_id: string, product_title: string, product_category: string, star_rating: int]

In [11]:
local_data = df.toPandas()

In [12]:
local_data.head()


,customer_id,product_id,product_title,product_category,star_rating
0,43638923,B0052SCU8U,AmazonBasics High Speed HDMI Cable,Electronics,4.0
1,43828045,B00A1EGOXM,AHB Chicago Pub Wall Table,Furniture,5.0
2,44326177,B007B5WHTE,Philips Bluetooth Soundbar Speaker with Subwoo...,Electronics,5.0
3,42024336,B00B25P27S,Emergency AM/FM/WX Crank Radio 20-576,Electronics,5.0
4,470490,B00VM5HIC0,Music for My Friends,Music,5.0


In [41]:
import pandas as pd

# Assuming 'df' is your DataFrame
# Convert columns to the appropriate types
local_data['customer_id'] = local_data['customer_id'].astype(int)

local_data['star_rating'] = local_data['star_rating'].astype(float)

# Show the first few rows to verify the changes
(local_data.head(20))

,customer_id,product_id,product_title,product_category,star_rating,product_id_encoded
0,43638923,B0052SCU8U,AmazonBasics High Speed HDMI Cable,Electronics,4.0,0
1,43828045,B00A1EGOXM,AHB Chicago Pub Wall Table,Furniture,5.0,1
2,44326177,B007B5WHTE,Philips Bluetooth Soundbar Speaker with Subwoo...,Electronics,5.0,2
3,42024336,B00B25P27S,Emergency AM/FM/WX Crank Radio 20-576,Electronics,5.0,3
4,470490,B00VM5HIC0,Music for My Friends,Music,5.0,4
5,15655104,B002HWR598,Convenience Concepts Designs2Go 3-Tier Wide TV...,Furniture,5.0,5
6,3024246,B00H9L7VIW,boostULTIMATE - 60 Capsules - Increase Workout...,Personal_Care_Appliances,5.0,6
7,40433972,B00LRXTW2Q,"Kirkland Signature - Creamy Almond Butter, 27 ...",Grocery,5.0,7
8,52264127,B002ZDOYMA,The Only Classic Country Collection You'll Ever,Music,3.0,8
9,20920671,0375412913,Lunar Park,Books,2.0,9


In [14]:
local_data['product_id_encoded'], _ = pd.factorize(local_data['product_id'])

# Show the first few rows to verify the changes
print(local_data.head())

# Optionally, check how many unique product IDs you have encoded
print("Unique product IDs:", local_data['product_id_encoded'].nunique())

   customer_id  product_id                                      product_title  \
0     43638923  B0052SCU8U                 AmazonBasics High Speed HDMI Cable   
1     43828045  B00A1EGOXM                         AHB Chicago Pub Wall Table   
2     44326177  B007B5WHTE  Philips Bluetooth Soundbar Speaker with Subwoo...   
3     42024336  B00B25P27S              Emergency AM/FM/WX Crank Radio 20-576   
4       470490  B00VM5HIC0                               Music for My Friends   

  product_category  star_rating  product_id_encoded  
0      Electronics          4.0                   0  
1        Furniture          5.0                   1  
2      Electronics          5.0                   2  
3      Electronics          5.0                   3  
4            Music          5.0                   4  
Unique product IDs: 134135


In [15]:
#Fill all Null values in
local_data['star_rating'] = local_data['star_rating'].fillna(0.0)


print(local_data['star_rating'].isnull().sum())

0


In [16]:
from datetime import datetime

In [17]:
class ALS:
  def __init__(self, client: Client):
    self.client = client

  def __preprocess_data(self, df, user_col, item_col, rating_col, chunk_size, n_factors):
    start_time = time.time()
    self.__print_status(0, 100, start_time, "Preprocessing data...")
    self.chunk_size = chunk_size
    self.n_factors = n_factors
    self.user_col = user_col
    self.item_col = item_col
    self.rating_col = rating_col

    self.u_ids = df[user_col].unique()
    self.i_ids = df[item_col].unique()

    self.u_mapping = { x: i for i, x in enumerate(self.u_ids) }
    self.i_mapping = { x: i for i, x in enumerate(self.i_ids) }
    df['u_encodings'] = df[user_col].map(self.u_mapping)
    df['i_encodings'] = df[item_col].map(self.i_mapping)
    self.__print_status(50, 100, start_time, "Preprocessing data...")

    self.n_users = len(self.u_ids)
    self.n_items = len(self.i_ids)
    self.n_ratings = df.shape[0]

    self.min_rating = np.min(df[rating_col])
    self.max_rating = np.max(df[rating_col])
    self.mean_rating = np.mean(df[rating_col])

    df = df[["u_encodings", "i_encodings", rating_col]]
    self.__print_status(100, 100, start_time, "Preprocessing data...")
    print()
    return df

  def __create_sparse_chunked_matrix(self, df):
    start_time = time.time()
    df_val = df.values
    sparse_df = sparse.COO(df_val[:, :2].T.astype(int), df_val[:, 2], shape=((self.n_users, self.n_items)))

    chunks = []
    for i in range(0, self.n_users, self.chunk_size):
      sub_chunks=[]
      self.__print_status(i + self.chunk_size, self.n_users, start_time, "Creating user-item matrix...")
      for j in range(0, self.n_items, self.chunk_size):
        sub_chunks.append(sparse_df[i: i + self.chunk_size, j: j + self.chunk_size])
      chunks.append(sub_chunks)

    self.__print_status(self.n_users, self.n_users, start_time, "Creating user-item matrix...")
    x = da.block(chunks)
    x_mask = da.sign(x).map_blocks(lambda x: x.todense(), dtype=np.ndarray) == 1
    print()

    return x, x_mask

  def __init_biases(self):
    u_biases = da.zeros((self.n_users, 1), chunks=(self.chunk_size,1))
    i_biases = da.zeros(self.n_items, chunks=(self.chunk_size,))
    return u_biases, i_biases

  def __init_latent_vectors(self):
    u_factors = da.random.uniform(0, 0.1, (self.n_users, self.n_factors), chunks=(self.chunk_size, self.n_factors))
    i_factors = da.random.uniform(0, 0.1, (self.n_items, self.n_factors), chunks=(self.chunk_size, self.n_factors))
    return u_factors, i_factors

  def __compute_learing_error(self, u_factors, i_factors, u_biases, i_biases, x, x_mask):
    pred = self.mean_rating + u_biases + u_factors @ i_factors.T + i_biases
    error = x - pred * x_mask
    return error

  def __get_training_errors(self, error):
    mae = da.sum(da.absolute(error)) / self.n_ratings
    mse = da.sum(error ** 2) / self.n_ratings
    rmse = da.sqrt(mse)
    return (mae, mse, rmse)

  def __plot_training_errors(self, errors):
    if not os.path.exists('res/'):
      os.mkdir('res/')

    mapped_errors = {
      "MAE": [],
      "MSE": [],
      "RMSE": [],
    }

    for error in errors:
      mapped_errors["MAE"].append(error[0])
      mapped_errors["MSE"].append(error[1])
      mapped_errors["RMSE"].append(error[2])

    sns.set_style("darkgrid")
    start_time = time.time()
    plt.figure()
    plt.subplots(figsize=(30, 5))
    for index, (error, error_values) in enumerate(mapped_errors.items()):
      self.__print_status(index + 1, len(mapped_errors), start_time, "Ploting training errors...")
      plt.subplot(130 + index + 1)
      plt.xlabel("Epoch", fontsize=24)
      plt.ylabel(error, fontsize=24)
      plt.plot(error_values)

    plt.savefig("res/{}-{}-{}.pdf".format(type(self).__name__, "training-errors", datetime.today().strftime('%Y-%m-%d-%H:%M:%S')))

    print()

  def __mae(self, a, b):
    return np.abs(np.subtract(a, b)).mean()

  def __mse(self, a, b):
    return np.square(np.subtract(a, b)).mean()

  def __rmse(self, a, b):
    return np.sqrt(((np.subtract(a, b))**2).mean())

  def __print_status(self, iter, max_iter, start_time, status, step=False):
    elapsed_time = time.time() - start_time
    bar_length = 70
    j = iter / max_iter
    sys.stdout.write('\r')
    if step:
      sys.stdout.write(f"[{'=' * int(bar_length * j):{bar_length}s}] {int(100 * j)}% Elapsed time: {round(elapsed_time, 3)} s - {status} ({iter}/{max_iter})")
    else:
      sys.stdout.write(f"[{'=' * int(bar_length * j):{bar_length}s}] {int(100 * j)}% Elapsed time: {round(elapsed_time, 3)} s - {status}")
    sys.stdout.flush()

  def fit(self,
          n_factors,
          train_df,
          chunk_size,
          epochs=50,
          lr=0.001,
          reg=0.001,
          collect_errors=False,
          plot_errors=False,
          user_col="customer_id",
          item_col="product_id",
          rating_col="star_rating",
  ):
    fit_start_time = time.time()
    df = self.__preprocess_data(train_df, user_col, item_col, rating_col, chunk_size, n_factors)
    x, x_mask = self.__create_sparse_chunked_matrix(df)
    u_biases, i_biases = self.__init_biases()
    u_factors, i_factors = self.__init_latent_vectors()

    start_time_epoch = time.time()
    train_errors = []
    error = self.__compute_learing_error(u_factors, i_factors, u_biases, i_biases, x, x_mask)
    for epoch in range(epochs):
      self.__print_status(epoch + 1, epochs, start_time_epoch, "Creating epochs", step=True)

      if collect_errors:
        train_errors.append(self.__get_training_errors(error))

      u_factors = u_factors + lr * (error @ i_factors - reg * u_factors)
      u_biases = u_biases + lr * da.sum(error - reg * u_biases, axis=1, keepdims=True)

      error = self.__compute_learing_error(u_factors, i_factors, u_biases, i_biases, x, x_mask)
      i_factors = i_factors + lr * ((u_factors.T @ error).T - reg * i_factors)
      i_biases = i_biases + lr * da.sum(error - reg * i_biases, axis=0, keepdims=True)

      error = self.__compute_learing_error(u_factors, i_factors, u_biases, i_biases, x, x_mask)

    print("\nComputing in parallel...")

    compute_start_time = time.time()
    if collect_errors:
      self.u_biases, self.i_biases, self.u_factors, self.i_factors, self.train_errors = compute(u_biases, i_biases, u_factors, i_factors, train_errors)
    else:
      self.u_biases, self.i_biases, self.u_factors, self.i_factors = compute(u_biases, i_biases, u_factors, i_factors)
    self.u_biases = self.u_biases.T
    compute_end_time = time.time()

    print("Compute parallel time: {} s".format(round(compute_end_time - compute_start_time, 3)))
    print("Compute parallel time per epoch: {} s".format(round((compute_end_time - compute_start_time) / epochs, 3)))
    print("Total fitting time: {} s".format(round(compute_end_time - fit_start_time, 3)))

    if plot_errors:
      self.__plot_training_errors(self.train_errors)

  def predict(self, test_df, user_col=None, item_col=None):
    if user_col is None: user_col = self.user_col
    if item_col is None: item_col = self.item_col

    predictions = []
    start_time = time.time()
    df = test_df[[user_col, item_col]].values
    df_len = len(df)

    for i in range(df_len):
      user, item = df[i][0], df[i][1]
      self.__print_status(i, df_len, start_time, "Predicting...")
      pred = self.mean_rating

      if user in self.u_mapping and item in self.i_mapping:
        u_id = self.u_mapping[user]
        i_id = self.i_mapping[item]

        pred += self.u_biases[0][u_id] + self.i_biases[0][i_id] + self.u_factors[u_id] @ self.i_factors[i_id]
        pred = min(max(self.min_rating, pred), self.max_rating)

      predictions.append(pred)
    print()

    return predictions

  def eval(self, ground_truths, predictions):
    mae = self.__mae(ground_truths, predictions)
    mse = self.__mse(ground_truths, predictions)
    rmse = self.__rmse(ground_truths, predictions)
    return mae, mse, rmse

In [18]:
from dask.distributed import Client




client = Client(n_workers=4)

train = local_data.sample(frac=0.7, random_state=7)
test = local_data.drop(train.index.tolist())

model = ALS(client)
model.fit(
        n_factors=30,
        train_df=train,
        epochs=5,
        chunk_size=5000,
        collect_errors=True,
        plot_errors=False
    )

predictions = model.predict(test)
gt = test["star_rating"].to_numpy()
eval = model.eval(gt, predictions)
print(eval)
client.shutdown()

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:35055
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:41787'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:36465'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:36919'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:35039'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:42349', name: 3, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:42349
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:58350
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:37881', name: 1, stat

[======================================================================] 100% Elapsed time: 0.446 s - Preprocessing data...
[======================================================================] 100% Elapsed time: 4.785 s - Creating user-item matrix...
[==============                                                        ] 20% Elapsed time: 0.012 s - Creating epochs (1/5)

/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 20
  out = blockwise(


[==========================================                            ] 60% Elapsed time: 2.303 s - Creating epochs (3/5)

/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 20
  out = blockwise(
/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 20
  out = blockwise(
/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 20
  out = blockwise(
/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 20
  out = blockwise(


...

/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 20
  out = blockwise(
/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 20
  out = blockwise(
/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 20
  out = blockwise(
/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 20
  out = blockwise(



Computing in parallel...


/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 20
  out = blockwise(
/usr/local/lib/python3.10/dist-packages/dask/array/routines.py:443: PerformanceWarning: Increasing number of chunks by factor of 20
  out = blockwise(


Compute parallel time: 4947.634 s
Compute parallel time per epoch: 989.527 s
Total fitting time: 4957.31 s
[===================================================================== ] 99% Elapsed time: 102.317 s - Predicting...
(0.9801092686544018, 1.5671413948641961, 1.2518551812666656)


INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:41787'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:36465'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:36919'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:35039'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:58368; closing.
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:58364; closing.
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:58340; closing.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://127.0.0.1:44367', name: 0, status: closing, memory: 0, pro

In [23]:
def recommend_products(model, user_id, N=10):
    # Check if user_id exists in the mapping
    if user_id not in model.u_mapping:
        return "User ID does not exist in the training data."

    user_index = model.u_mapping[user_id]

    # Compute scores using dot product of user factors with all item factors
    user_factors = model.u_factors[user_index]
    scores = user_factors @ model.i_factors.T + model.i_biases[0]

    # Get the top N items with the highest scores
    top_items_indices = np.argsort(-scores)[:N]  # Get indices of top scores

    # Map back to item IDs
    top_items_ids = [model.i_ids[idx] for idx in top_items_indices]

    return top_items_ids


In [48]:
# Example usage
user_id = 40433972 # example user ID from your dataset
top_products = recommend_products(model, user_id, N=20)
print("Top 10 recommended products for user {}: {}".format(user_id, top_products))


Top 10 recommended products for user 40433972: ['B00H9L7VIW', 'B00HES9CMS', 'B0006VJ6TO', 'B00P6TUO5G', 'B0016BFR4G', 'B00HXXO332', 'B00OYRW4UE', 'B0002JG2NI', 'B003L1ZYYM', 'B003CDXJUK', 'B008LQZP6E', 'B002C9MSAC', 'B00K504UUG', 'B006Z921ES', 'B013CSAV0U', 'B00Y1C9770', '0895260174', 'B0002DMANU', 'B00UCFVIDQ', 'B000YDDF6O']


In [49]:
# Sample recommended product IDs
recommended_product_ids = top_products

# Convert to DataFrame for easier merging
recommended_df = pd.DataFrame(recommended_product_ids, columns=['product_id'])


In [53]:
results = pd.merge(recommended_df, local_data[['product_id', 'product_title']], on='product_id', how='left').drop_duplicates()
results.head(20)

,product_id,product_title
0,B00H9L7VIW,boostULTIMATE - 60 Capsules - Increase Workout...
3391,B00HES9CMS,Viva Naturals #1 Best Selling Certified Organi...
4240,B0006VJ6TO,Body Back Company’s Body Back Buddy Trigger Po...
4862,B00P6TUO5G,"Viva Naturals Organic Non-GMO Cacao Powder, 2 ..."
5009,B0016BFR4G,Uncle Lee's Organic Green Tea -- 100 Tea Bags ...
5174,B00HXXO332,"Pulse Oximeter, Blood Oxygen Monitor"
5500,B00OYRW4UE,Elite Sportz Exercise Sliders are Double Sided...
5655,B0002JG2NI,Home Health Castor Oil
5785,B003L1ZYYM,AmazonBasics High-Speed HDMI Cable - 6.5 Feet ...
5881,B003CDXJUK,Hearing Aid Battery Powerone size 10 made in G...
